## Image Enhancement Code

This code notebook accompanies the JCURA 2024 poster "Mountains of Confusion: Evaluating Image Enhancement to Improve AI Landscape Classification" by Larissa Bron and is Method #2. 

All coding was completed with ChatGPT. 

### Prepare images for enhancements 

Are they corrupted? Are they all the same type of file? Are the historic and repeat the same size?

1. Check if images in a folder are not being read as images (are they corrupted?). 

In [ ]:
import os
from PIL import Image

# Path to the folder containing the images
folder_path = "/path/to/your/folder"

# Create a subfolder to store corrupt images
corrupt_folder = os.path.join(folder_path, "CorruptImages")
os.makedirs(corrupt_folder, exist_ok=True)

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    # Create the file path
    file_path = os.path.join(folder_path, filename)

    try:
        # Open and load the image
        with Image.open(file_path) as img:
            img.load()  # Load the image data
            print(f"{filename} is a valid image.")
    except (IOError, SyntaxError) as e:
        # Get the file extension
        file_extension = os.path.splitext(filename)[1].lower()

        if file_extension in (".jpg", ".jpeg", ".tif", ".tiff", ".png", ".bmp", ".gif"):
            print(f"{filename} is corrupt or not a valid image: {str(e)}")
            
            # Move the corrupt image to the "CorruptImages" subfolder
            new_file_path = os.path.join(corrupt_folder, filename)
            os.rename(file_path, new_file_path)
        else:
            print(f"{filename} is not an image. It is a {file_extension[1:].upper()} file.")


2. Convert a folder of images to grayscale, uint8, and .jpg so that all image enhancements can be applied.

In [ ]:
import os
import numpy as np
from PIL import Image

def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff'))

# Path to the folder containing the input images
input_folder = "/path/to/your/folder"

# Path to the folder where the output images will be saved
output_folder = "/path/to/your/folder"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get the list of files in the input folder
file_list = os.listdir(input_folder)

# Loop through each file in the input folder
for file_name in file_list:
    if not is_image_file(file_name):
        print(f"Ignored non-image file: {file_name}")
        continue  # Ignore non-image files

    # Get the full path of the input file
    input_path = os.path.join(input_folder, file_name)

    # Open the image using Pillow
    image = Image.open(input_path)

    # Check if the image is not already in uint8 format
    if image.mode != 'L' and image.mode != 'I;16':
        # Convert the image to grayscale
        image = image.convert('L')
        print(f"Converted to grayscale: {file_name}")
    elif image.mode == 'I;16':
        # Convert 16-bit grayscale image to uint8
        image = np.array(image)
        image = (image / 256).astype('uint8')
        image = Image.fromarray(image)
        print(f"Converted to uint8: {file_name}")

    # Save the image to the output folder as JPEG
    new_file_name = os.path.splitext(file_name)[0] + "_GrayUInt8.jpg"
    output_path = os.path.join(output_folder, new_file_name)
    image.save(output_path, "JPEG")
    print(f"Saved as {new_file_name}")


3. Confirm that all images are now grayscale, uint8, and .jpg. 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Folder path containing the images
folder_path = '/path/to/your/folder'

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', 'tif', 'tiff'))]

# Iterate over each image file
for image_file in image_files:
    # Read the image file using matplotlib
    image_path = os.path.join(folder_path, image_file)
    image = plt.imread(image_path)
    
    # Convert the image to a NumPy array
    image_array = np.array(image)
    
    # Print the array and data type
    print(f'Image: {image_file}')
    print(f'Shape: {image_array.shape}')
    print(f'Data Type: {image_array.dtype}')
    print()


4. Confirm that original images and converted images are the same size in pixels. 

In [ ]:
from PIL import Image
import os

# Path to the two folders containing the images
input_folder = "/path/to/your/folder"
output_folder = "/path/to/your/folder"

# Get the list of files in the input folder
input_files = os.listdir(input_folder)
output_files = os.listdir(output_folder)

# Compare images in the two folders
for input_filename in input_files:
    if input_filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
        input_basename = os.path.splitext(input_filename)[0]  # Get base name without extension
        output_filename = input_basename + "_GrayUInt8.jpg"

        if output_filename in output_files:
            input_path = os.path.join(input_folder, input_filename)
            output_path = os.path.join(output_folder, output_filename)

            input_img = Image.open(input_path)
            output_img = Image.open(output_path)

            input_size = input_img.size  # Dimensions in pixels (width x height)
            output_size = output_img.size  # Dimensions in pixels (width x height)

            print(f"Comparison for: {input_basename}")
            print(f"Input Size (Dimensions): {input_size} pixels (Width x Height)")
            print(f"Output Size (Dimensions): {output_size} pixels (Width x Height)")
            print("-" * 20)


### Image Enhancements - Contrast

Histograms are provided to help visualize the changes that contrast enhancement effect on each image. 

### Contrast 1: CLAHE

OpenCV CLAHE: https://docs.opencv.org/4.x/d5/daf/tutorial_py_histogram_equalization.html

Adjustments that ChatGPT suggested:

"#clipLimit: This parameter determines the contrast limiting threshold. A lower value restricts the contrast enhancement to a smaller range, resulting in a more conservative enhancement. Increasing the value allows for a wider range of enhancement, potentially increasing the overall contrast.

#tileGridSize: This parameter sets the size of the grid for histogram equalization. A smaller grid size can capture local details more effectively but may result in a more uneven enhancement. Conversely, a larger grid size can provide a smoother enhancement across the image but may miss fine details."

1. Trying different parameter adjustments and reviewing the output to see which most consistently improved the images.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Set the path to your image folder
folder_path = '/path/to/your/folder'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Define the combinations of Clip Limit and Tile Grid Size to try
combinations = [
    (2.0, (4, 4)),
    (4.0, (8, 8)),
    (6.0, (4, 4)),
    (8.0, (6, 6)),
]

# Iterate through the files in the folder
for file_name in file_list:
    # Check if the file is a supported image format
    if file_name.lower().endswith(('.jpg', '.tif', '.tiff', '.jpeg')):
        # Load the image
        image_path = os.path.join(folder_path, file_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Create a new figure for each image
        fig, axs = plt.subplots(2, len(combinations) + 1, figsize=(15, 8))

        # Set the title as the image filename
        fig.suptitle(file_name, fontsize=16)

        # Display the original image
        axs[0, 0].imshow(image, cmap='gray')
        axs[0, 0].set_title('Original Image')
        axs[0, 0].axis('off')

        # Display the histogram of the original image
        axs[1, 0].hist(image.ravel(), bins=256, range=[0, 256], color='gray', alpha=0.6)
        axs[1, 0].set_title('Histogram (Original Image)')
        axs[1, 0].set_xlim([0, 256])

        # Apply CLAHE with different combinations and show the results
        for i, (clip_limit, tile_grid_size) in enumerate(combinations, start=1):
            # Apply CLAHE with the current combination
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
            enhanced_image = clahe.apply(image)

            # Display the enhanced image with the current parameters
            axs[0, i].imshow(enhanced_image, cmap='gray')
            axs[0, i].set_title(f'Clip Limit: {clip_limit},\nTile Grid Size: {tile_grid_size}')
            axs[0, i].axis('off')

            # Display the histogram of the enhanced image
            axs[1, i].hist(enhanced_image.ravel(), bins=256, range=[0, 256], color='gray', alpha=0.6)
            axs[1, i].set_title(f'Histogram (Clip Limit: {clip_limit})')
            axs[1, i].set_xlim([0, 256])

        # Adjust the spacing between subplots
        plt.tight_layout(rect=[0, 0, 1, 0.95])

        # Show the figure for each image
        plt.show()


2. Apply CLAHE with the chosen clip limit = 2.0 and tile grid size of 8x8 to a folder of images.

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Set the path to your image folder
folder_path = '/path/to/your/folder'

# Create a new folder for the CLAHE images
output_folder = os.path.join(folder_path, 'CLAHE')
os.makedirs(output_folder, exist_ok=True)

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Supported image extensions
supported_extensions = ('.jpg', '.jpeg', '.png', '.tif', '.tiff')

# Define spacing between rows
row_spacing = 1.0

# Iterate through the files in the folder
for file_name in file_list:
    # Check if the file has an image extension
    if any(file_name.lower().endswith(ext) for ext in supported_extensions):
        # Load the image
        image_path = os.path.join(folder_path, file_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Apply CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_image = clahe.apply(image)

        # Create a figure for each row
        fig, axs = plt.subplots(2, 2, figsize=(12, 8))

        # Display the original image on the left
        axs[0, 0].imshow(image, cmap='gray')
        axs[0, 0].set_title('Original Image')
        axs[0, 0].axis('off')

        # Display the enhanced image on the right
        axs[0, 1].imshow(enhanced_image, cmap='gray')
        axs[0, 1].set_title('Enhanced Image (CLAHE)')
        axs[0, 1].axis('off')

        # Display histograms below the images
        axs[1, 0].hist(image.ravel(), bins=256, range=(0, 256), density=True, color='gray')
        axs[1, 0].set_title('Histogram (Original)')
        axs[1, 0].set_xlim([0, 255])
        axs[1, 0].set_ylim([0, 0.03])

        axs[1, 1].hist(enhanced_image.ravel(), bins=256, range=(0, 256), density=True, color='gray')
        axs[1, 1].set_title('Histogram (Enhanced)')
        axs[1, 1].set_xlim([0, 255])
        axs[1, 1].set_ylim([0, 0.03])

        # Adjust spacing between rows
        plt.subplots_adjust(hspace=row_spacing)

        # Show the file names above and below the row
        title_text = os.path.splitext(file_name)[0]
        fig.suptitle(title_text, fontsize=14, y=1.02)

        plt.show()

        # Save the enhanced image with modified name
        output_file_name = os.path.splitext(file_name)[0] + '_CLAHE' + os.path.splitext(file_name)[1]
        output_path = os.path.join(output_folder, output_file_name)
        cv2.imwrite(output_path, enhanced_image)
        print(f"Enhanced image saved: {output_path}")


### Contrast 2: Gamma Correction from Power Law Transformation

OpenCV Gamma Correction: https://docs.opencv.org/3.4/d3/dc1/tutorial_basic_linear_transform.html

ChatGPT suggested adjusting parameters by "Trying multiple different exponents for gamma and comparing within the window, followed by applying my favourite version. ChatGPT suggested: The gamma value affects the overall brightness and contrast of the image. A gamma value less than 1 will darken the image and increase contrast, while a gamma value greater than 1 will brighten the image and decrease contrast. For example:Gamma < 1: Darkens the image (e.g., gamma = 0.5), Gamma = 1: No change (linear correction), Gamma > 1: Brightens the image (e.g., gamma = 1.5)."

1. Compare different amounts of gamma correction applied to a folder of images.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Set the path to the folder containing the input images
image_folder = '/path/to/your/folder'

# Define the gamma values to try
gamma_values = [0.5, 0.75, 1.5, 1.75]

# Get the list of image files in the folder
image_files = [filename for filename in os.listdir(image_folder) if filename.lower().endswith('.jpg')]

# Iterate through the image files
for image_filename in image_files:
    image_path = os.path.join(image_folder, image_filename)

    # Load the image as grayscale (uint8 format)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Create a new figure for each image
    fig, axs = plt.subplots(2, len(gamma_values) + 1, figsize=(15, 6))

    # Set the title as the image filename
    fig.suptitle(image_filename, fontsize=16)

    # Display the original image
    axs[0, 0].imshow(image, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 0].axis('off')

    # Display the histogram of the original image
    axs[1, 0].hist(image.ravel(), bins=256, range=[0, 256], color='gray', alpha=0.6)
    axs[1, 0].set_title('Histogram (Original Image)')
    axs[1, 0].set_xlim([0, 256])

    # Iterate through the gamma values and perform gamma correction
    for i, gamma in enumerate(gamma_values):
        # Perform gamma correction
        gamma_corrected_image = np.power(image / 255.0, gamma) * 255.0
        gamma_corrected_image = np.clip(gamma_corrected_image, 0, 255).astype(np.uint8)

        # Display the gamma-corrected image
        axs[0, i+1].imshow(gamma_corrected_image, cmap='gray')
        axs[0, i+1].set_title(f'Gamma = {gamma}')
        axs[0, i+1].axis('off')

        # Display the histogram of the gamma-corrected image
        axs[1, i+1].hist(gamma_corrected_image.ravel(), bins=256, range=[0, 256], color='gray', alpha=0.6)
        axs[1, i+1].set_title(f'Histogram (Gamma = {gamma})')
        axs[1, i+1].set_xlim([0, 256])

    # Adjust the spacing between subplots and heights of rows
    plt.tight_layout(rect=[0, 0, 1, 0.9], h_pad=0.5)

    # Show the figure for each image
    plt.show()


2. Apply favourite gamma of 0.75 to the folder of images and save the output images.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to perform gamma correction
def gamma_correction(image, gamma):
    return np.power(image / 255.0, gamma) * 255.0

# Set the path to the original image folder
original_folder = '/path/to/your/folder'

# Create the output folder for gamma-corrected images
output_folder = os.path.join(original_folder, 'gamma0.75')
os.makedirs(output_folder, exist_ok=True)

# Supported image extensions
supported_extensions = ('.jpg', '.jpeg', '.png', '.tif', '.tiff')

# Get a list of all files in the original image folder
file_list = os.listdir(original_folder)

# Iterate through the files in the original image folder
for file_name in file_list:
    # Check if the file has an image extension
    if any(file_name.lower().endswith(ext) for ext in supported_extensions):
        # Load the original image
        original_image_path = os.path.join(original_folder, file_name)
        original_image = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)

        # Apply gamma correction with gamma=0.75
        gamma_strength = 0.75
        gamma_corrected_image = gamma_correction(original_image, gamma_strength)

        # Create a figure to compare the images and histograms
        fig, axs = plt.subplots(2, 2, figsize=(12, 10))

        # Plot the original image
        axs[0, 0].imshow(original_image, cmap='gray')
        axs[0, 0].set_title('Original Image')
        axs[0, 0].axis('off')

        # Plot the histogram of the original image
        axs[1, 0].hist(original_image.ravel(), bins=256, range=(0, 256), color='gray', alpha=0.7)
        axs[1, 0].set_title('Histogram (Original Image)')

        # Plot the gamma-corrected image
        axs[0, 1].imshow(gamma_corrected_image, cmap='gray')
        axs[0, 1].set_title(f'Gamma Corrected (Gamma={gamma_strength})')
        axs[0, 1].axis('off')

        # Plot the histogram of the gamma-corrected image
        axs[1, 1].hist(gamma_corrected_image.ravel(), bins=256, range=(0, 256), color='gray', alpha=0.7)
        axs[1, 1].set_title(f'Histogram (Gamma Corrected, Gamma={gamma_strength})')

        # Adjust the layout and show the figure
        plt.tight_layout()
        plt.show()

        # Save the gamma-corrected image with modified name
        output_file_name = os.path.splitext(file_name)[0] + f'_gamma{gamma_strength}' + os.path.splitext(file_name)[1]
        output_path = os.path.join(output_folder, output_file_name)
        cv2.imwrite(output_path, gamma_corrected_image)

        print(f"Gamma-corrected image saved: {output_path}")


### Contrast 3: Contrast Stretch

Used a version of contrast stretching that ChatGPT devised. 

Parameter adjustment suggested by ChatGPT was "the parameter num_bins controls the number of bins used in the histogram calculation. You can adjust this parameter to change the granularity of contrast stretching.

For example, if you increase num_bins, you will have more bins to distribute the pixel values across, potentially leading to a finer adjustment. Conversely, decreasing num_bins will result in fewer bins and a coarser adjustment."

1. Compare the effect of adjusting the number of bins. 

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def contrast_stretching(image, num_bins=256):
    # Calculate the histogram of the input image
    hist, bin_edges = np.histogram(image, bins=num_bins, range=(0, 255))

    # Calculate the cumulative distribution function (CDF) of the histogram
    cdf = hist.cumsum()

    # Scale the CDF to be in the range [0, 255]
    cdf_normalized = cdf * 255 / cdf[-1]

    # Use linear interpolation of the CDF to equalize the image
    equalized_image = np.interp(image, bin_edges[:-1], cdf_normalized)

    return equalized_image

# Folder paths for input
input_folder = '/path/to/your/folder'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all the files in the input folder
file_list = os.listdir(input_folder)

# Define the list of bin values to compare
bin_values = [256, 512, 128]

for filename in file_list:
    # Skip any non-image files
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    # Load the grayscale image using Pillow
    image_path = os.path.join(input_folder, filename)
    image = Image.open(image_path).convert('L')

    # Convert the Pillow image to a NumPy array
    image_np = np.array(image)

    # Create subplots for each image
    fig, axs = plt.subplots(2, len(bin_values) + 1, figsize=(15, 8))

    # Plot the original image
    axs[0, 0].imshow(image, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 0].axis('off')

    # Plot the histogram of the original image
    axs[1, 0].hist(image_np.ravel(), bins=256, range=(0, 255), color='gray', alpha=0.6)
    axs[1, 0].set_title('Original Histogram')
    axs[1, 0].set_xlim([0, 255])

    # Iterate over different bin values and perform bihistogram equalization
    for i, num_bins in enumerate(bin_values, start=1):
        equalized_image_np = contrast_stretching(image_np, num_bins=num_bins)

        # Convert the equalized NumPy array back to a Pillow image
        equalized_image = Image.fromarray(equalized_image_np).convert('L')  # Convert back to 8-bit grayscale

        # Plot the equalized image
        axs[0, i].imshow(equalized_image, cmap='gray')
        axs[0, i].set_title(f'Contrast Stretched (Bins: {num_bins})')
        axs[0, i].axis('off')

        # Plot the histogram of the equalized image
        axs[1, i].hist(equalized_image_np.ravel(), bins=num_bins, range=(0, 255), color='gray', alpha=0.6)
        axs[1, i].set_title(f'Contrast Stretching (Bins: {num_bins})')
        axs[1, i].set_xlim([0, 255])

    plt.tight_layout()
    plt.show()

print('Review of contrast stretching results displayed.')


2. Contrast stretching for a folder of images. Chose 256 bins since the output images all looked similar.

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def contrast_stretching(image, num_bins=256):
    # Calculate the histogram of the input image
    hist, bin_edges = np.histogram(image, bins=num_bins, range=(0, 255))

    # Calculate the cumulative distribution function (CDF) of the histogram
    cdf = hist.cumsum()

    # Scale the CDF to be in the range [0, 255]
    cdf_normalized = cdf * 255 / cdf[-1]

    # Use linear interpolation of the CDF to equalize the image
    equalized_image = np.interp(image, bin_edges[:-1], cdf_normalized)

    return equalized_image

# Folder paths for input and output
input_folder = '/path/to/your/folder'
output_folder = '/path/to/your/folder'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all the files in the input folder
file_list = os.listdir(input_folder)

for filename in file_list:
    # Skip any non-image files
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    # Load the grayscale image using Pillow
    image_path = os.path.join(input_folder, filename)
    image = Image.open(image_path).convert('L')

    # Convert the Pillow image to a NumPy array
    image_np = np.array(image)

    # Perform bihistogram equalization
    equalized_image_np = bihistogram_equalization(image_np)

    # Convert the equalized NumPy array back to a Pillow image
    equalized_image = Image.fromarray(equalized_image_np).convert('L')  # Convert back to 8-bit grayscale

    # Save the equalized image with "_Bihist" appended to the filename
    output_filename = os.path.splitext(filename)[0] + '_ContrastStretch' + os.path.splitext(filename)[1]
    output_path = os.path.join(output_folder, output_filename)
    equalized_image.save(output_path)

    # Display the original and equalized images with file name
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(image, cmap='gray')
    axs[0].set_title(f'Original Image: {filename}')
    axs[0].axis('off')

    axs[1].imshow(equalized_image, cmap='gray')
    axs[1].set_title(f'Contrast Stretched: {output_filename}')
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()

print("Contrast stretched images saved to the output folder.")


### Image Enhancements - Denoising

### Denoising 1: Non-local Means

OpenCV non-local means denoising: https://docs.opencv.org/3.4/d1/d79/group__photo__denoise.html#ga4c6b0031f56ea3f98f768881279ffe93

There were no parameters that could be compared and adjusted for this kind of denoising. 

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Input folder path containing images
input_folder = '/path/to/your/folder'

# Output folder name for saving denoised images
output_folder_name = 'Denoised/'

# List of supported image extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# Zoomed-in region parameters
zoom_size = 200  # Size of the zoomed-in region

# Iterate over each file in the input folder
for filename in os.listdir(input_folder):
    # Check if the file has an image extension
    _, ext = os.path.splitext(filename)
    if ext.lower() not in image_extensions:
        print(f"Ignoring non-image file: {filename}")
        continue

    # Load the image
    image = cv2.imread(os.path.join(input_folder, filename))

    if image is None:
        print(f"Error: Unable to read image: {filename}")
        continue
    
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform Non-local Means Denoising with default parameters
    denoised_image = cv2.fastNlMeansDenoising(image_gray, None)

    # Generate the output filename
    output_filename = os.path.splitext(filename)[0] + '_Denoised.jpg'

    # Create the output folder within the input folder if it doesn't exist
    output_folder = os.path.join(input_folder, output_folder_name)
    os.makedirs(output_folder, exist_ok=True)

    # Save the denoised image in the output folder
    cv2.imwrite(os.path.join(output_folder, output_filename), denoised_image)

    # Get the center region of the denoised image
    h, w = denoised_image.shape
    center_row = h // 2
    center_col = w // 2
    zoomed_region = denoised_image[
        center_row - zoom_size // 2: center_row + zoom_size // 2,
        center_col - zoom_size // 2: center_col + zoom_size // 2
    ]

    # Create a figure to display the comparison
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the original image
    axs[0].imshow(image[..., ::-1])  # Convert BGR to RGB for displaying
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    # Plot the center region of the denoised image
    axs[1].imshow(zoomed_region, cmap='gray')
    axs[1].set_title('Center of Denoised Image')
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()

    print(f"Denoised {filename} using basic Non-local Means Denoising.")
    print(f"Denoised image saved: {os.path.join(output_folder, output_filename)}")


### Denoising 2: Bilateral Filter

OpenCV Bilateral Filter: https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html

Notes: I didn't try different values for the parameters for this denoising and comparing the effects of denoising on the images was easier in Affinity Photo external to this Jupyter notebook. 

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define input and output directories
input_folder = '/path/to/your/folder'
output_folder = '/path/to/your/folder'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Denoising parameters
diameter = 9        # Diameter of each pixel neighborhood
sigma_color = 75   # Filter sigma in color space
sigma_space = 75   # Filter sigma in coordinate space

# Loop through images in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg')):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        # Load the image in grayscale
        image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
        
        # Apply Bilateral Filter for denoising
        denoised_image = cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)
        
        # Save the denoised image
        cv2.imwrite(output_path, denoised_image)
        
        print(f"Denoised {filename} and saved to {output_path}")
        
        # Display the original and denoised images
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(image, cmap='gray')
        plt.title('Original Image')
        
        plt.subplot(1, 2, 2)
        plt.imshow(denoised_image, cmap='gray')
        plt.title('Denoised Image')
        
        plt.tight_layout()
        plt.show()


### Denoising 3: Wavelet denoising

This version of Wavelet Denoising was chosen by ChatGPT. I would suggest following the scikit-image version instead here: https://scikit-image.org/docs/stable/auto_examples/filters/plot_denoise_wavelet.html. 

I did not trial different variations of parameter adjustment for this denoising. Suggestion from ChatGPT was "to try adjusting the threshold value, which determines how aggressively the wavelet coefficients are thresholded to remove noise. Lowering the threshold will result in more aggressive noise removal, but it might also lead to loss of some fine details in the image."

Effects of denoising on images were better viewed in Affinity Photo external to this Jupyter notebook. 

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt

# Define input and output folders
input_folder = "/path/to/your/folder"
output_folder = "/path/to/your/folder"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Define wavelet denoising function
def denoise_image(image):
    original_shape = image.shape

    # Pad the image to make its dimensions divisible by 2
    target_shape = (original_shape[0] + original_shape[0] % 2, original_shape[1] + original_shape[1] % 2)
    padded_image = np.pad(image, ((0, target_shape[0] - original_shape[0]), (0, target_shape[1] - original_shape[1])), mode='constant')

    coeffs = pywt.dwt2(padded_image, 'haar')
    cA, (cH, cV, cD) = coeffs

    threshold = 20  # Adjust this threshold as needed

    cA_denoised = pywt.threshold(cA, threshold)
    cH_denoised = pywt.threshold(cH, threshold)
    cV_denoised = pywt.threshold(cV, threshold)
    cD_denoised = pywt.threshold(cD, threshold)

    denoised_coeffs = (cA_denoised, (cH_denoised, cV_denoised, cD_denoised))
    denoised_padded_image = pywt.idwt2(denoised_coeffs, 'haar')

    # Crop the denoised image back to the original dimensions
    denoised_image = denoised_padded_image[:original_shape[0], :original_shape[1]]

    return denoised_image.astype(np.uint8)

# Process images and save denoised images
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg')):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        denoised_image = denoise_image(image)

        output_filename = os.path.splitext(filename)[0] + "_waveletdenoised.jpg"
        output_path = os.path.join(output_folder, output_filename)

        cv2.imwrite(output_path, denoised_image)

        # Display original and denoised images in the notebook
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(image, cmap='gray')
        plt.title("Original Image")

        plt.subplot(1, 2, 2)
        plt.imshow(denoised_image, cmap='gray')
        plt.title("Denoised Image")

        plt.tight_layout()
        plt.show()
        
        


### Image Enhancements - Sharpening

The effects of sharpening are best viewed zoomed-in to an image.

### Sharpening 1: Unsharp Mask

Scikit-image Unsharp Masking: https://scikit-image.org/docs/stable/auto_examples/filters/plot_unsharp_mask.html.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, img_as_float
from skimage.filters import gaussian, unsharp_mask

# Define input and output folders
input_folder = '/path/to/your/folder'
output_folder = '/path/to/your/folder'

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Parameters for unsharp masking
amount = 2.0  # Adjust this value as needed
radius = 1

# Process images in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg')):
        image_path = os.path.join(input_folder, filename)
        original_image = io.imread(image_path)
        
        # Convert the image to floating-point format in the range [0, 1]
        image_float = img_as_float(original_image)

        # Create a mask highlighting the edges using a Gaussian filter
        smoothed_image = gaussian(image_float, sigma=1)
        edge_mask = image_float - smoothed_image

        # Unsharp Masking on the edges only
        sharpened_edges = unsharp_mask(edge_mask, radius=radius, amount=amount)

        # Blend the sharpened edges with the original image
        sharpened_image = image_float + sharpened_edges
        sharpened_image = np.clip(sharpened_image, 0, 1)  # Ensure the values are within [0, 1]
        sharpened_image = (sharpened_image * 255).astype(np.uint8)

        # Create a new filename with '_unsharpmask' and the amount added to the end
        output_filename = os.path.splitext(filename)[0] + f'_unsharpmask_amount_{amount}.jpg'
        output_path = os.path.join(output_folder, output_filename)

        # Save the sharpened image
        io.imsave(output_path, sharpened_image)

        # Display the original and sharpened images
        plt.figure(figsize=(10, 5))
        
        # Display filename at the top of the row
        plt.suptitle(f'Original Image: {filename}', fontsize=12, y=1.02)
        
        plt.subplot(1, 2, 1)
        plt.imshow(original_image, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(sharpened_image, cmap='gray')
        plt.title(f'Unsharp Masked (Amount={amount})')
        plt.axis('off')

        plt.tight_layout()
        plt.show()


### Sharpening 2: High Pass Filter

Pillow High Pass Filter: https://pillow.readthedocs.io/en/stable/reference/ImageFilter.html

Code below uses a technique that ChatGPT suggested in which first there is an unsharp mask applied, then the sharpening filter. 

In [ ]:
import os
from PIL import Image, ImageFilter
from matplotlib import pyplot as plt

# Function to apply high pass filter and sharpen a grayscale image
def sharpen_image(image):
    sharpened = image.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))
    return sharpened

# Path to the folder containing the input grayscale images
input_folder = "/path/to/your/folder"

# Specify the output folder for sharpened images
output_folder = "/path/to/your/folder"
os.makedirs(output_folder, exist_ok=True)

# Allowed file extensions
allowed_extensions = [".jpg", ".jpeg"]

# Loop through each image in the input folder
for filename in os.listdir(input_folder):
    file_ext = os.path.splitext(filename)[1].lower()
    
    # Check if the file extension is allowed
    if file_ext in allowed_extensions:
        # Load the image
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path).convert("L")
        
        # Apply high pass filter and sharpen the image
        sharpened = sharpen_image(image)
        
        # Save the sharpened image with a new name
        new_filename = os.path.splitext(filename)[0] + "_HighSharp" + file_ext
        output_path = os.path.join(output_folder, new_filename)
        sharpened.save(output_path)
        
        # Display the original and sharpened images with filename
        fig, ax = plt.subplots(1, 2, figsize=(10, 6))
        
        # Plot the original grayscale image
        ax[0].imshow(image, cmap='gray')
        ax[0].set_title(f"Original: {filename}")
        ax[0].axis('off')
        
        # Plot the sharpened grayscale image
        ax[1].imshow(sharpened, cmap='gray')
        ax[1].set_title("Sharpened")
        ax[1].axis('off')
        
        # Show the plot
        plt.tight_layout()
        plt.show()


### Sharpening 3: Wiener Filter 

Scikit-image Wiener Filter: https://scikit-image.org/docs/stable/auto_examples/filters/plot_restoration.html

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, img_as_float
from skimage.restoration import wiener

# Define input and output folders
input_folder = '/path/to/your/folder'
output_folder = '/path/to/your/folder'

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Parameters for Wiener filter
psf = np.ones((5, 5)) / 25.0  # Example point spread function
noise_var = 0.1  # Example noise variance

# Process images in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg')):
        image_path = os.path.join(input_folder, filename)
        original_image = io.imread(image_path, as_gray=True)
        
        # Convert the image to floating-point format
        image_float = img_as_float(original_image)
        
        # Apply Wiener filter
        restored_image = wiener(image_float, psf, noise_var)
        
        # Convert back to uint8 and rescale
        restored_image = (restored_image * 255).astype(np.uint8)
        
        # Save the restored image
        output_path = os.path.join(output_folder, filename)
        io.imsave(output_path, restored_image)

        # Display the original and restored images with filename
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        
        # Plot the original grayscale image
        ax[0].imshow(original_image, cmap='gray')
        ax[0].set_title(f'Original: {filename}')
        ax[0].axis('off')
        
        # Plot the restored grayscale image
        ax[1].imshow(restored_image, cmap='gray')
        ax[1].set_title('Restored Image (Wiener)')
        ax[1].axis('off')
        
        # Show the plot
        plt.tight_layout()
        plt.show()
